In [1]:
import datetime
import time
import pytz
import pandas as pd

from itertools import chain
from datetime import timedelta, date, datetime, timezone
from basketball_reference_web_scraper import client

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

from secrets import *

In [2]:
dbname = 'cluj'
engine = create_engine('postgres://%s:%s@localhost/%s'%('docker','docker',dbname))

if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [3]:
#https://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

### Get Schedules

In [4]:
schedule = client.season_schedule(season_end_year=2020)

In [ ]:
#start = schedule[0]['start_time']
#end = datetime.now(pytz.timezone('US/Central')) - timedelta(days=1)
#start, end

In [11]:
schedule_df = pd.DataFrame(schedule)
schedule_df['away_team'] = schedule_df.away_team.apply(lambda x: x.name)
schedule_df['home_team'] = schedule_df.home_team.apply(lambda x: x.name)
schedule_df['start_time'] = schedule_df.start_time.dt.tz_convert('US/Central')
schedule_df['start_time'] = schedule_df.start_time.apply(lambda x: x.tz_localize(None))
schedule_df['start_date'] = schedule_df.start_time.apply(lambda x: x.date())
schedule_df['season_end_year'] = 2020
schedule_df.head()

,away_team,away_team_score,home_team,home_team_score,start_time,start_date,season_end_year
0,NEW_ORLEANS_PELICANS,122.0,TORONTO_RAPTORS,130.0,2019-10-22 19:00:00,2019-10-22,2020
1,LOS_ANGELES_LAKERS,102.0,LOS_ANGELES_CLIPPERS,112.0,2019-10-22 21:30:00,2019-10-22,2020
2,CHICAGO_BULLS,125.0,CHARLOTTE_HORNETS,126.0,2019-10-23 18:00:00,2019-10-23,2020
3,DETROIT_PISTONS,119.0,INDIANA_PACERS,110.0,2019-10-23 18:00:00,2019-10-23,2020
4,CLEVELAND_CAVALIERS,85.0,ORLANDO_MAGIC,94.0,2019-10-23 18:00:00,2019-10-23,2020


In [12]:
schedule_df.to_sql('nba_schedule', con=engine, if_exists='replace', index=False)

In [13]:
for year in range(2001, 2020):
    print(year)
    schedule = client.season_schedule(season_end_year=year)
    schedule_df = pd.DataFrame(schedule)
    schedule_df['away_team'] = schedule_df.away_team.apply(lambda x: x.name)
    schedule_df['home_team'] = schedule_df.home_team.apply(lambda x: x.name)
    schedule_df['start_time'] = schedule_df.start_time.dt.tz_convert('US/Central')
    schedule_df['start_time'] = schedule_df.start_time.apply(lambda x: x.tz_localize(None))
    schedule_df['start_date'] = schedule_df.start_time.apply(lambda x: x.date())
    schedule_df['season_end_year'] = year
    schedule_df.to_sql('nba_schedule', con=engine, if_exists='append', index=False)

2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


### Get a bunch of boxscores

In [ ]:
# I downloaded season_end_year 2019 and 2020. Does not include playoffs.

In [ ]:
boxscores_list = []
for day in daterange(start, end):
    print(day)
    boxscores = client.player_box_scores(day=day.day, month=day.month, year=day.year)
    for item in boxscores:
        item.update( {"date":datetime.strftime(day.date(), format = '%Y-%m-%d')})
    boxscores_list.append(boxscores)
    time.sleep(2)

In [ ]:
boxscores_df = pd.DataFrame(list(chain.from_iterable(boxscores_list)))
boxscores_df['rebounds'] = boxscores_df.offensive_rebounds + boxscores_df.defensive_rebounds
boxscores_df.rename(columns={'attempted_field_goals':'fga', 'attempted_free_throws':'fta',
                            'made_three_point_field_goals':'threes', 'made_field_goals':'fgm',
                            'made_free_throws':'ftm'}, inplace=True)
boxscores_df['twos'] = boxscores_df.fgm - boxscores_df.threes
boxscores_df['points'] = (boxscores_df.threes * 3) + (boxscores_df.twos * 2) + (boxscores_df.ftm * 1)
boxscores_df.drop(columns=['attempted_three_point_field_goals','defensive_rebounds','offensive_rebounds',
                          'game_score','slug','turnovers','outcome','twos','personal_fouls','location'], inplace=True)
boxscores_df['opponent'] = boxscores_df.opponent.apply(lambda x: x.name)
boxscores_df['team'] = boxscores_df.team.apply(lambda x: x.name)
boxscores_df.columns.values

In [ ]:
boxscores_df.to_sql('boxscores', con=engine, if_exists='append', index=False)